In [ ]:
# %% Calculus 2 - Section 6.51
#    CalculArt: create "music" from equations

# This code pertains to a calculus course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/pycalc2_x
# The code in this repository is developed to solve the exercises provided along
# the course, and it has been written partially indepentently and partially
# from the code developed by the course instructor.


In [2]:
import numpy             as np
import sympy             as sym
import matplotlib.pyplot as plt
import scipy.integrate   as spi
import math
import mpmath

from scipy.signal                     import find_peaks
from IPython.display                  import display,Math
from google.colab                     import files
from IPython.display                  import Audio
from scipy.io                         import wavfile
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')

import matplotlib.animation as animation
from matplotlib import rc
rc('animation', html='jshtml')


In [ ]:
# %% CalculArt 1a
#    Make a simple sound

# sampling rate (44.1 kHz) and time
fs = 44100
t  = np.arange(0,3-1/fs,1/fs)

freq1 = 261.6   # middle C
freq2 = 329.628 # E

# Create the sound (C+E)
sound = np.sin(2*np.pi*freq1*t) + np.sin(2*np.pi*freq2*t)

# Play
Audio(sound,rate=fs)


In [ ]:
# %% CalculArt 1b
#    Generate left and right channels

# Left and right channels (right channel is AM)
left  = 1*np.sin(2*np.pi*freq1*t)
right = np.sin(2*np.pi*freq2*t) * np.sin(2*np.pi*2*t)

# Play
Audio([left,right],rate=fs)


In [ ]:
# %% CalculArt 1c
#    Scale the data to [-1,1]

# Scale mono
mono2write = (sound-np.min(sound))*2 / (np.max(sound)-np.min(sound)) - 1
print(np.min(sound),np.max(sound))
print(np.min(mono2write),np.max(mono2write))

# Scale stereo (it needs to be a numpy array of size [samples X channels])
stereo       = np.vstack((left,right)).T
stereo2write = (stereo-np.min(stereo))*2 / (np.max(stereo)-np.min(stereo)) - 1

# Write to wav file
wavfile.write('audio1_mono_codechallenge_51_calculart_1.wav',fs,mono2write)
wavfile.write('audio2_stereo_codechallenge_51_calculart_1.wav',fs,stereo2write)

files.download('audio1_mono_codechallenge_51_calculart_1.wav')
files.download('audio2_stereo_codechallenge_51_calculart_1.wav')


In [ ]:
# %% CalculArt 2
#    Make AM (amplitude-modulated) and FM (frequency-modulated) sounds; in AM
#    the volume changes over time, in FM the pitch changes over time

# AM
t = np.arange(-np.pi,np.pi,1/fs)

a     = 1.5
AM    = (1-a)+np.cos(a*t) + np.log(np.abs(t)+.001)
AM    = 2*(AM-np.min(AM)) / (np.max(AM)-np.min(AM))
sound = AM * np.sin(2*np.pi*440*t)

plt.plot(t,AM);

# Save
wavfile.write('audio3_am_codechallenge_51_calculart_2.wav',fs,sound)
files.download('audio3_am_codechallenge_51_calculart_2.wav')

# Play it
Audio(sound,rate=fs)


In [ ]:
# %% CalculArt 2
#    Continue ...

# FM
fc = 440  # carrier frequency
bw = 75   # bandwidth

# Modulating signal
modsig = np.sin(2*np.pi*5*t)

# Integrate the modulating signal and scale
modint = bw * np.cumsum(modsig) / fs

# FM signal
sound = np.cos(2*np.pi*(fc*t + modint))

# Save
wavfile.write('audio4_fm_codechallenge_51_calculart_2.wav',fs,sound)
files.download('audio4_fm_codechallenge_51_calculart_2.wav')

# Play
Audio(sound,rate=fs)


In [ ]:
# %% CalculArt 3
#    Mathy art noise

# FM part
FM = np.sin(2*np.pi*1.5*t) + np.log(np.abs(t)*2)
modint = bw * np.cumsum(FM) / fs

# AM part
AM = 1-np.linspace(-1,1,len(t))**2

# Sound
sound = AM * np.cos(2*np.pi*(fc*t + modint))
display(Audio(sound,rate=fs))

# visualize the FM modulation
phi = (1 + np.sqrt(5)) / 2
_,axs = plt.subplots(1,2,figsize=(1.5*phi*5,5))
axs[0].plot(t,FM,'k',linewidth=2)
axs[0].set(xlabel='Time (s)',xlim=t[[0,-1]],yticks=[],ylabel='Frequency modulation (a.u.)',title='FM component')

axs[1].plot(t,AM,'k',linewidth=2)
axs[1].set(xlabel='Time (s)',xlim=t[[0,-1]],yticks=[],ylabel='Amplitude modulation (a.u.)',title='AM component')

plt.tight_layout()

plt.savefig('fig22_codechallenge_51_calculart_3.png')
plt.show()
files.download('fig22_codechallenge_51_calculart_3.png')

# Save
wavfile.write('audio5_codechallenge_51_calculart_3.wav',fs,sound)
files.download('audio5_codechallenge_51_calculart_3.wav')


In [ ]:
# %% CalculArt 4
#    Have fun !

# Time
t = np.arange(-3*np.pi,3*np.pi,1/fs)

# FM part
FM = np.sin(2*np.pi*1*t) + np.sin(0.8*np.pi*0.3*t) + np.log(np.abs(t)*2) + np.sin(4*np.pi*5*t)
modint = bw * np.cumsum(FM) / fs

# AM part
AM = 1-np.linspace(-1,1,len(t))**2

# Sound
sound = AM * np.cos(2*np.pi*(fc*t + modint))
display(Audio(sound,rate=fs))

# Save
wavfile.write('audio6_codechallenge_51_calculart_4.wav',fs,sound)
files.download('audio6_codechallenge_51_calculart_4.wav')
